In [289]:
import pandas as pd
import numpy as np

In [290]:
BMJ_data_all__b_new = pd.read_csv('../input/BMJ-data-all--b-new.csv', sep='\t')
item_profiles1 = pd.read_csv('../input/item-profiles1.csv', sep=';')
item_profiles2 = pd.read_csv('../input/item-profiles2.csv', sep=';')
item_profiles3 = pd.read_csv('../input/item-profiles3.csv', sep=';')
ratings = pd.read_csv('../input/user-item-rating.csv', sep='\t', names=['user_id','item_id','rating'])

In [291]:
item_rating_count = ratings['item_id'].value_counts(normalize=False, sort=True, ascending=True)
item_rating_count

346      20
1924     20
2957     20
4445     20
1171     20
       ... 
134     290
140     291
50      308
137     321
58      358
Name: item_id, Length: 1031, dtype: int64

In [292]:
ratingsExeptItemProfiles

,user_id,item_id,rating


In [293]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = ratings.copy()
y = ratings['user_id']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y random_state=42)

SyntaxError: invalid syntax (<ipython-input-293-6119aa5a239d>, line 9)

In [294]:
#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [295]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['user_id'], X_test['item_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, item) for (user, item) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [296]:
#Define the baseline model to always return 3.
def baseline(user_id, item_id):
    return 3.0

In [297]:
score(baseline)

1.6173239383771754

In [298]:
#Import the required classes and methods from the surprise library
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate



In [299]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\marti/.surprise_data/ml-100k
196 ['603', '480', '64', '479', '963', '318', '408', '496', '654', '170']
186 ['318', '483', '496', '272', '50', '479', '114', '1137', '750', '64']
22 ['408', '315', '12', '603', '64', '169', '23', '135', '357', '646']
244 ['127', '134', '474', '23', '187', '611', '511', '603', '192', '285']
166 ['169', '199', '318', '50', '483', '513', '1137', '705', '64', '89']
298 ['12', '64', '173', '513', '520', '313', '480', '272', '515', '169']
115 ['134', '408', '169', '513', '474', '285', '175', '179', '168', '135']
253 ['174', '272', '69', '172', '178', '498', '143', '181', '513', '191']
305 ['515', '603', '498', '23', '647', '919', '136', '657', '589', '493']
6 ['654', '923', '443', '652', '963', '171', '603', '705', '114', '694']
62 ['169', '317', '654'

In [300]:
from collections import defaultdict
from surprise.model_selection import KFold
from sklearn.metrics import ndcg_score

def precision_recall_ndcg_at_k(predictions):
    """Return precision and recall at k metrics for each user"""
    
    k = 10
    threshold = 4
    
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:k]
        
    
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
    predictions_list = [[x[3] for x in predictions]]
    true_scores = [[x[2] for x in predictions]]
    
    
    precision = (sum(prec for prec in precisions.values())) / len(precisions)
    recall = (sum(rec for rec in recalls.values())) / len(recalls)
    ndcg = ndcg_score(true_scores, predictions_list, k)
    
    return precision, recall, ndcg, top_n
    

In [317]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold

#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter
data = Dataset.load_from_df(ratings, reader)

# We'll use the famous SVD algorithm.
svd = SVD()

kf = KFold(n_splits=5)

evaluations

for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)
    precision, recall, ndcg, top_n = precision_recall_ndcg_at_k(predictions)
    
    print("Precision:", precision, "Recall:", recall, "NDCG_score:", ndcg)

C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8742867783285754 Recall: 0.8321554761272777 NDCG_score: 0.9766561514195581


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8679709682664751 Recall: 0.8302817526751154 NDCG_score: 0.980811808118081


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8660975396199977 Recall: 0.8288780461297187 NDCG_score: 0.9770392749244712


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8706887911702422 Recall: 0.8313599354491935 NDCG_score: 0.9711475409836062
Precision: 0.8684774172550191 Recall: 0.8314370027204732 NDCG_score: 0.9665573770491801


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [302]:
predictions

[Prediction(uid=78848, iid=4568, r_ui=5.0, est=4.712358574932885, details={'was_impossible': False}),
 Prediction(uid=51646, iid=315, r_ui=5.0, est=3.96558156906943, details={'was_impossible': False}),
 Prediction(uid=71756, iid=903, r_ui=5.0, est=4.501656957054494, details={'was_impossible': False}),
 Prediction(uid=37323, iid=454, r_ui=5.0, est=4.3054377876619006, details={'was_impossible': False}),
 Prediction(uid=78972, iid=2737, r_ui=4.0, est=4.273918894496909, details={'was_impossible': False}),
 Prediction(uid=51752, iid=418, r_ui=5.0, est=4.04334403933466, details={'was_impossible': False}),
 Prediction(uid=52812, iid=28, r_ui=5.0, est=4.758247870832595, details={'was_impossible': False}),
 Prediction(uid=33285, iid=748, r_ui=5.0, est=4.754328868561773, details={'was_impossible': False}),
 Prediction(uid=59348, iid=761, r_ui=5.0, est=4.329298402429772, details={'was_impossible': False}),
 Prediction(uid=20663, iid=257, r_ui=4.0, est=4.425771697153498, details={'was_impossible':

In [318]:
top_n

defaultdict(list,
            {72892: [(2041, 4.709980903712825),
              (472, 4.639758535245688),
              (322, 4.547556466746882),
              (2359, 4.534879354903443),
              (134, 4.4902722156999655),
              (58, 4.483580222367397),
              (609, 4.477301947075829),
              (1041, 4.458072228673179),
              (652, 4.431679966113037),
              (532, 4.368714033644504)],
             81196: [(555, 4.7852957347744765),
              (2442, 4.66844212373896),
              (532, 4.528387237139981),
              (1119, 4.51130838328058),
              (1691, 4.5035861198050755),
              (117, 4.5013232649918535),
              (394, 4.4461760993657125),
              (1020, 4.439126701667912),
              (2975, 4.418720332957095),
              (1080, 4.413690080207114)],
             8298: [(980, 5),
              (1052, 5),
              (2352, 5),
              (885, 4.920456996508598),
              (206, 4.9138285404573

In [304]:
recalls

0.8332970356992068

In [305]:
ndcg_score

<function sklearn.metrics._ranking.ndcg_score(y_true, y_score, *, k=None, sample_weight=None, ignore_ties=False)>

In [306]:
def healthiness(itemsDataframe):
    
    fat = ((itemsDataframe['Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    sugar = ((itemsDataframe['Sugar (g)'] * 4) / itemsDataframe['Calories (kCal)']) * 100
    saturatedFat = ((itemsDataframe['Saturated Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    
    itemsDataframe['Unhealtiness'] = (fat / 3) + sugar + saturatedFat
    
    itemsDataframe.loc[fat > 30, 'fatPoints'] = 0
    itemsDataframe.loc[fat >= 40, 'fatPoints'] = -1
    itemsDataframe.loc[fat >= 50, 'fatPoints'] = -2
    itemsDataframe.loc[fat >= 60, 'fatPoints'] = -3
    itemsDataframe.loc[fat >= 70, 'fatPoints'] = -4
    itemsDataframe.loc[fat >= 80, 'fatPoints'] = -50
    itemsDataframe.loc[fat <= 30, 'fatPoints'] = 1 
    itemsDataframe.loc[fat <= 20, 'fatPoints'] = 2
    itemsDataframe.loc[fat <= 10, 'fatPoints'] = 3
    itemsDataframe.loc[fat <= 5, 'fatPoints'] = 4 
    itemsDataframe.loc[fat <= 1, 'fatPoints'] = 5

    itemsDataframe.loc[sugar > 10, 'sugarPoints'] = 0
    itemsDataframe.loc[sugar >= 13, 'sugarPoints'] = -1
    itemsDataframe.loc[sugar >= 16, 'sugarPoints'] = -2
    itemsDataframe.loc[sugar >= 19, 'sugarPoints'] = -3
    itemsDataframe.loc[sugar >= 22, 'sugarPoints'] = -4
    itemsDataframe.loc[sugar >= 25, 'sugarPoints'] = -50
    itemsDataframe.loc[sugar <= 10, 'sugarPoints'] = 1 
    itemsDataframe.loc[sugar <= 7, 'sugarPoints'] = 2
    itemsDataframe.loc[sugar <= 5, 'sugarPoints'] = 3 
    itemsDataframe.loc[sugar <= 3, 'sugarPoints'] = 4
    itemsDataframe.loc[sugar <= 1, 'sugarPoints'] = 5 
        
    itemsDataframe.loc[saturatedFat > 10, 'satFatPoints'] = 0
    itemsDataframe.loc[saturatedFat >= 13, 'satFatPoints'] = -1
    itemsDataframe.loc[saturatedFat >= 16, 'satFatPoints'] = -2
    itemsDataframe.loc[saturatedFat >= 19, 'satFatPoints'] = -3
    itemsDataframe.loc[saturatedFat >= 22, 'satFatPoints'] = -4
    itemsDataframe.loc[saturatedFat >= 25, 'satFatPoints'] = -50
    itemsDataframe.loc[saturatedFat <= 10, 'satFatPoints'] = 1 
    itemsDataframe.loc[saturatedFat <= 7, 'satFatPoints'] = 2
    itemsDataframe.loc[saturatedFat <= 5, 'satFatPoints'] = 3 
    itemsDataframe.loc[saturatedFat <= 3, 'satFatPoints'] = 4
    itemsDataframe.loc[saturatedFat <= 1, 'satFatPoints'] = 5 
        
       
    itemsDataframe['Healthiness'] = itemsDataframe['fatPoints'] + itemsDataframe['satFatPoints'] + itemsDataframe['sugarPoints']

In [307]:
item_profiles = item_profiles1.set_index('Recipe ID').join(item_profiles2.set_index('Recipe ID'), how= 'inner')#.join(item_profiles3.set_index('Recipe ID'), how= 'inner')
healthiness(item_profiles)
item_profiles

,Category,Directions,Name,Fiber (g),Sodium (g),Carbohydrates (g),Fat (g),Protein (g),Sugar (g),Saturated Fat (g),...,Average Rating,Average Sentiment,Number of Ratings,Number of Bookmarks,Year of Publishing,Unhealtiness,fatPoints,sugarPoints,satFatPoints,Healthiness
Recipe ID,,,,,,,,,,,,,,,,,,,,,
1939,Home > Recipes > Main Dish > Roasts,"In a medium bowl, stir together the cranberry ...",Slow Cooker Cranberry Pork,1.1,0.18,32.9,15.1,26.8,31.2,3.6,...,4.23,1.86,146,2150,2000,54.144385,0.0,-50.0,1.0,-49.0
771,Home > Recipes > Main Dish > Stuffed Mai...,"On large plate, combine 1 tablespoon flour, sa...",Cousin Cosmo's Greek Chicken,0.8,0.69,4.8,10.2,31.0,1.5,3.9,...,4.45,1.97,117,2676,2002,30.000000,0.0,4.0,-1.0,3.0
3783,Home > Recipes > Main Dish > Steaks and ...,"In a blender, combine the soy sauce, honey, vi...",Barbequed Marinated Flank Steak,0.1,0.66,9.7,27.8,24.8,8.9,6.3,...,4.49,2.19,85,1829,2000,45.283505,-3.0,1.0,-1.0,-3.0
205,Home > Recipes > Main Dish > Burgers > ...,Preheat grill for high heat.<br>In a large bow...,Juiciest Hamburgers Ever,0.5,0.20,9.1,17.8,21.5,1.7,7.0,...,4.39,1.51,216,3912,2002,42.777778,-2.0,4.0,-3.0,-1.0
1136,Home > Recipes > Meat and Poultry > Chic...,Season both sides of chicken breasts with garl...,Braised Balsamic Chicken,1.2,0.51,7.6,7.0,23.8,5.4,1.3,...,4.36,1.98,707,10217,2000,27.704082,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Home > Recipes > Main Dish > Pasta,"In a large saute pan, heat oil, butter and gar...",Creamy Chicken on Linguine,2.3,0.38,45.4,24.8,30.4,3.8,13.4,...,4.38,1.83,94,3715,2000,40.191205,-1.0,4.0,-4.0,-1.0
168,Home > Recipes > Main Dish > Roasts,"Cut up potatoes, onions, and celery in to fair...",Easiest Pot Roast Ever,6.2,0.55,42.4,25.3,31.7,4.5,10.0,...,4.51,1.96,98,1517,2000,34.961977,-1.0,3.0,-2.0,0.0
1847,Home > Recipes > Main Dish > Steaks and ...,Preheat grill for high heat.<br>Use a fork to ...,Beer and Brown Sugar Steak Marinade,0.1,0.55,9.2,20.8,38.1,7.9,8.3,...,4.38,1.96,112,1984,2003,43.256410,-1.0,1.0,-3.0,-3.0


In [363]:
top_k

defaultdict(list,
            {72892: [(2041, 4.709980903712825),
              (472, 4.639758535245688),
              (322, 4.547556466746882),
              (2359, 4.534879354903443),
              (134, 4.4902722156999655),
              (58, 4.483580222367397),
              (609, 4.477301947075829),
              (1041, 4.458072228673179),
              (652, 4.431679966113037),
              (532, 4.368714033644504)],
             81196: [(555, 4.7852957347744765),
              (2442, 4.66844212373896),
              (532, 4.528387237139981),
              (1119, 4.51130838328058),
              (1691, 4.5035861198050755),
              (117, 4.5013232649918535),
              (394, 4.4461760993657125),
              (1020, 4.439126701667912),
              (2975, 4.418720332957095),
              (1080, 4.413690080207114)],
             8298: [(980, 5),
              (1052, 5),
              (2352, 5),
              (885, 4.920456996508598),
              (206, 4.9138285404573

In [362]:
top_k = top_n.copy()

In [377]:
for x in top_n.items():
    for y in x[1]:
        print(y[1])


4.709980903712825
4.639758535245688
4.547556466746882
4.534879354903443
4.4902722156999655
4.483580222367397
4.477301947075829
4.458072228673179
4.431679966113037
4.368714033644504
4.7852957347744765
4.66844212373896
4.528387237139981
4.51130838328058
4.5035861198050755
4.5013232649918535
4.4461760993657125
4.439126701667912
4.418720332957095
4.413690080207114
5
5
5
4.920456996508598
4.913828540457392
4.831035246726041
4.830956848497923
4.812520441489818
4.809870505820286
4.800128882049856
5
5
4.971581458812937
4.964467333447591
4.955140042569756
4.936588613229968
4.868421447236211
4.831857324207364
4.829009072532329
4.80166064484593
5
4.8889024098437295
4.817678985705393
4.785781657270051
4.785573194286548
4.461601041965802
4.458153580898372
4.389732710441019
5
5
5
5
4.914792140046517
4.860008004961159
4.649769498032818
4.584546943406737
4.569419034288755
4.7639736543390905
4.746284580424218
4.7149993899405915
4.673219153606154
4.668847875697454
4.567335330241591
4.567035263260643
4.5

4.4936842668779144
4.491030308820101
4.470710867869941
4.465024871924603
4.444872620811229
4.43982224303001
4.422776253706025
4.420363126303981
4.974856494651608
4.797567721643666
4.7164908200688735
4.706862431879797
4.659355308654187
4.650860723009014
4.549348337440796
4.468685135031014
4.396226310044942
4.323092070554625
4.243259315148026
4.224783747404916
4.1538983668263025
4.071679362267079
4.052645306515319
4.043334766116422
3.933072775309069
3.9110038600949704
4.832803298190929
4.775555965721166
4.7092106585468665
4.654818042989954
4.619036616454357
4.2149604634675955
4.175548427551644
4.008564056037312
3.9721646159154513
3.9362589026219044
3.8147547418491
4.780480239231613
4.530076742563713
4.523085547429598
3.9176885229271408
4.3851635877454385
4.252459191451813
4.212537895883045
4.100330784573211
3.8980921395591053
4.830658254984642
4.728859557527399
4.542490131611029
4.516195954348628
4.491142078704263
4.269654454106141
4.165624676986077
4.122470167689024
4.092831892202322
4.

3.508522796570709
3.4928838648067932
4.828019142104979
4.819524297449186
4.734465908192912
4.356670997291527
4.250991341027641
4.877019231356779
4.855219298739712
4.821687764304527
4.671282154197069
4.432083937293923
4.332541805204025
4.315751926751072
4.227759348616693
4.186153176839538
5
4.972283396202084
4.9417803581369935
4.75640187553654
4.636325187506596
4.740827276737844
4.626398067252044
4.496875527114923
4.493367864906801
4.489724557012977
4.483034304251561
4.42884510525151
4.405239377837519
4.392900726648654
4.3831276781544695
4.713665267448552
4.6055912915696755
4.596476769067551
4.468708063228617
4.465984679727047
4.395268150017158
4.387265726682489
4.235421134348739
4.223257430803791
4.921654252254116
4.763157536796862
4.639999484044773
4.579221905203578
4.573350766622894
4.499618950339854
4.493191230177645
4.213255768019225
4.175891506708992
4.120850694450334
4.911775974179209
4.432167956908743
4.297992143084762
4.749480770281119
4.641389561085013
4.619779830275295
4.5765

4.572813123556431
4.463840452888731
4.121912150334479
4.300294847182917
4.201629368822559
4.1107685699292835
4.077544568627332
4.034149447497181
3.7053480487684056
4.719646736531941
4.662070876932832
4.5383106962757775
4.533738402170784
4.392310589438247
4.365107821450246
4.134432168041508
4.127604508040677
4.119061228844195
3.9797815328045134
4.500201293822042
4.433112310239455
4.382472850256023
4.3108324245724425
4.308093390215618
4.28518370018398
4.2580956362105695
4.101279448845711
4.090644628929506
3.9695685643661873
4.6151996026784206
4.371372323123183
4.359862095978739
4.328909493967284
4.204350934489283
4.197432389376392
5
5
4.985074066907545
4.785115030123573
4.685217183139876
4.661022744233916
4.5047352160066465
4.7711539024974785
4.648550673821465
4.993925043046164
4.969260530602286
4.963845388654744
4.859073161220829
4.733711768388663
4.6295565912482886
4.597591392052406
4.5783729153076
4.498840190819839
4.494205986006168
4.476380715298675
4.462390201956122
4.40276814975847

TypeError: 'numpy.float64' object is not iterable

In [360]:
top_k

defaultdict(list,
            {72892: [(2041, 4.709980903712825),
              (472, 4.639758535245688),
              (322, 4.547556466746882),
              (2359, 4.534879354903443),
              (134, 4.4902722156999655),
              (58, 4.483580222367397),
              (609, 4.477301947075829),
              (1041, 4.458072228673179),
              (652, 4.431679966113037),
              (532, 4.368714033644504)],
             81196: [(555, 4.7852957347744765),
              (2442, 4.66844212373896),
              (532, 4.528387237139981),
              (1119, 4.51130838328058),
              (1691, 4.5035861198050755),
              (117, 4.5013232649918535),
              (394, 4.4461760993657125),
              (1020, 4.439126701667912),
              (2975, 4.418720332957095),
              (1080, 4.413690080207114)],
             8298: [(980, 5),
              (1052, 5),
              (2352, 5),
              (885, 4.920456996508598),
              (206, 4.9138285404573

In [348]:
top_k.values()

dict_values([[(2041, 4.709980903712825), (472, 4.639758535245688), (322, 4.547556466746882), (2359, 4.534879354903443), (134, 4.4902722156999655), (58, 4.483580222367397), (609, 4.477301947075829), (1041, 4.458072228673179), (652, 4.431679966113037), (532, 4.368714033644504)], [(555, 4.7852957347744765), (2442, 4.66844212373896), (532, 4.528387237139981), (1119, 4.51130838328058), (1691, 4.5035861198050755), (117, 4.5013232649918535), (394, 4.4461760993657125), (1020, 4.439126701667912), (2975, 4.418720332957095), (1080, 4.413690080207114)], [(980, 5), (1052, 5), (2352, 5), (885, 4.920456996508598), (206, 4.913828540457392), (2074, 4.831035246726041), (63, 4.830956848497923), (19, 4.812520441489818), (331, 4.809870505820286), (173, 4.800128882049856)], [(513, 5), (203, 5), (58, 4.971581458812937), (566, 4.964467333447591), (63, 4.955140042569756), (1296, 4.936588613229968), (807, 4.868421447236211), (724, 4.831857324207364), (2055, 4.829009072532329), (656, 4.80166064484593)], [(676, 5

In [310]:
for x in top_n.items():
    for y in x[1]:
        print(item_profiles.at[y[0], 'Healthiness'])

#item_profiles

-4.0
-5.0
-1.0
7.0
-2.0
-7.0
-50.0
-3.0
-5.0
5.0
-7.0
3.0
5.0
1.0
-3.0
0.0
-1.0
-47.0
-48.0
4.0
-54.0
2.0
-46.0
-48.0
1.0
-54.0
-48.0
-1.0
-2.0
-45.0
5.0
4.0
-6.0
-1.0
-2.0
-52.0
-1.0
4.0
-1.0
-50.0
-3.0
-49.0
-3.0
3.0
-49.0
5.0
-48.0
-2.0
-49.0
-1.0
1.0
-3.0
-1.0
-48.0
-1.0
5.0
-2.0
-2.0
7.0
0.0
5.0
-54.0
-9.0
0.0
0.0
0.0
-2.0
4.0
1.0
5.0
-3.0
3.0
-48.0
-49.0
0.0
1.0
-48.0
-1.0
-2.0
5.0
4.0
-1.0
-1.0
5.0
0.0
3.0
6.0
-1.0
-49.0
-48.0
-3.0
0.0
-1.0
-51.0
-48.0
7.0
2.0
-4.0
1.0
2.0
6.0
-1.0
2.0
-2.0
-48.0
-48.0
8.0
5.0
9.0
-49.0
0.0
7.0
-1.0
-4.0
6.0
1.0
-1.0
-49.0
-2.0
-48.0
-48.0
-48.0
-48.0
-3.0
-49.0
0.0
1.0
-2.0
-3.0
5.0
9.0
-2.0
-49.0
-50.0
-3.0
5.0
-54.0
-1.0
0.0
-49.0
5.0
5.0
1.0
8.0
-5.0
2.0
-46.0
-3.0
-48.0
-3.0
-48.0
8.0
-50.0
-49.0
5.0
-1.0
-50.0
-51.0
-1.0
5.0
-50.0
-7.0
7.0
-1.0
-1.0
-45.0
2.0
0.0
-50.0
2.0
-1.0
1.0
-2.0
0.0
-51.0
-47.0
1.0
-48.0
-47.0
0.0
-48.0
-49.0
2.0
-52.0
-2.0
1.0
0.0
1.0
-2.0
-48.0
2.0
-49.0
8.0
1.0
3.0
7.0
-3.0
-3.0
1.0
0.0
0.0
-2.0
-49.0
4.0
-7.0
1

-2.0
-54.0
-3.0
-1.0
-4.0
5.0
8.0
1.0
-50.0
-1.0
-3.0
4.0
-49.0
5.0
-4.0
5.0
0.0
0.0
-8.0
-51.0
2.0
-7.0
6.0
-49.0
2.0
-48.0
5.0
-3.0
-3.0
-45.0
-5.0
-47.0
6.0
1.0
3.0
5.0
0.0
-9.0
3.0
5.0
-2.0
-8.0
-49.0
-3.0
5.0
1.0
1.0
-2.0
2.0
-2.0
-2.0
2.0
1.0
-50.0
2.0
1.0
-1.0
-5.0
4.0
8.0
4.0
-45.0
-4.0
6.0
2.0
-1.0
-4.0
2.0
-43.0
-9.0
-49.0
-2.0
-45.0
5.0
2.0
-48.0
-47.0
-47.0
-48.0
-4.0
0.0
-2.0
5.0
-48.0
6.0
2.0
-49.0
-45.0
-45.0
5.0
-2.0
-9.0
6.0
-46.0
-47.0
-4.0
-1.0
-48.0
0.0
-54.0
-48.0
2.0
2.0
1.0
-9.0
-2.0
-5.0
-2.0
1.0
-7.0
-2.0
-3.0
0.0
-8.0
-49.0
-48.0
5.0
-1.0
2.0
-54.0
-49.0
-1.0
-8.0
4.0
2.0
-49.0
-2.0
2.0
-6.0
8.0
-2.0
-50.0
-1.0
4.0
-6.0
6.0
-47.0
-7.0
-1.0
4.0
-49.0
-3.0
5.0
6.0
13.0
-7.0
-8.0
0.0
4.0
4.0
-3.0
-3.0
-2.0
-55.0
-48.0
-48.0
2.0
-1.0
2.0
7.0
3.0
-1.0
0.0
4.0
-49.0
-2.0
7.0
-49.0
2.0
8.0
8.0
-49.0
1.0
-49.0
8.0
5.0
-3.0
-49.0
4.0
-3.0
9.0
-47.0
4.0
5.0
5.0
1.0
0.0
-3.0
0.0
1.0
-2.0
3.0
-8.0
0.0
6.0
-2.0
5.0
-4.0
-7.0
-50.0
7.0
-3.0
3.0
-8.0
0.0
3.0
2.0
-7.0
-49.0
5

1.0
0.0
3.0
9.0
1.0
3.0
-48.0
-1.0
-3.0
0.0
-3.0
-2.0
-8.0
1.0
0.0
-3.0
-3.0
0.0
-3.0
-46.0
-1.0
0.0
-9.0
2.0
2.0
3.0
-50.0
2.0
7.0
0.0
1.0
-1.0
-1.0
8.0
-3.0
11.0
-6.0
0.0
3.0
7.0
-50.0
-3.0
0.0
0.0
-1.0
-49.0
2.0
11.0
2.0
4.0
-1.0
-49.0
1.0
8.0
-1.0
-2.0
-47.0
4.0
-51.0
1.0
-6.0
-48.0
8.0
4.0
6.0
1.0
0.0
-47.0
-1.0
2.0
-45.0
-8.0
2.0
-3.0
0.0
-47.0
3.0
3.0
-7.0
-52.0
2.0
2.0
1.0
-2.0
0.0
-1.0
0.0
3.0
1.0
3.0
-3.0
1.0
-48.0
1.0
-4.0
-2.0
-2.0
-4.0
-1.0
-49.0
-2.0
-3.0
-1.0
-45.0
-6.0
-1.0
5.0
-3.0
-52.0
-49.0
6.0
-2.0
-49.0
4.0
1.0
-48.0
-47.0
5.0
-47.0
8.0
-46.0
-7.0
0.0
-49.0
6.0
2.0
3.0
-3.0
-2.0
1.0
-9.0
-48.0
0.0
-46.0
-50.0
-3.0
-1.0
2.0
3.0
-7.0
4.0
-5.0
2.0
-1.0
0.0
-48.0
5.0
7.0
-48.0
4.0
2.0
-49.0
-47.0
4.0
-50.0
-1.0
-52.0
-1.0
-2.0
0.0
3.0
4.0
-2.0
-49.0
-1.0
11.0
1.0
2.0
5.0
-7.0
-3.0
-7.0
-48.0
-1.0
-2.0
3.0
1.0
1.0
-48.0
-3.0
-52.0
-50.0
3.0
1.0
-1.0
-53.0
-52.0
-1.0
-52.0
1.0
-1.0
-2.0
0.0
3.0
-5.0
-46.0
-1.0
-7.0
4.0
4.0
-1.0
-48.0
0.0
0.0
2.0
0.0
-48.0
7.0
-49.0
5.0


-3.0
5.0
2.0
-51.0
-3.0
-6.0
-3.0
-3.0
0.0
-6.0
-1.0
0.0
0.0
-1.0
-1.0
2.0
4.0
-50.0
0.0
0.0
6.0
4.0
1.0
13.0
-1.0
0.0
0.0
-4.0
-1.0
1.0
-1.0
0.0
6.0
-3.0
-1.0
-7.0
-7.0
-1.0
1.0
0.0
-9.0
-47.0
-48.0
-49.0
5.0
-1.0
-46.0
-48.0
1.0
1.0
-51.0
4.0
2.0
4.0
3.0
-2.0
-3.0
3.0
-6.0
-49.0
-5.0
-49.0
4.0
6.0
0.0
-3.0
4.0
2.0
-48.0
-46.0
5.0
-43.0
-2.0
-49.0
4.0
2.0
5.0
-3.0
4.0
-6.0
-49.0
-48.0
0.0
-3.0
-6.0
1.0
-51.0
1.0
4.0
5.0
0.0
-1.0
-48.0
-1.0
1.0
-3.0
-3.0
0.0
-1.0
5.0
-1.0
5.0
0.0
-2.0
13.0
-45.0
-46.0
0.0
2.0
0.0
0.0
-1.0
-2.0
-5.0
2.0
-54.0
-46.0
-3.0
-3.0
-48.0
-7.0
-7.0
-4.0
1.0
-49.0
-47.0
-46.0
-52.0
1.0
5.0
-2.0
1.0
0.0
-48.0
0.0
6.0
1.0
0.0
1.0
0.0
2.0
-49.0
-47.0
-49.0
-3.0
-48.0
2.0
-2.0
-1.0
-52.0
4.0
0.0
4.0
-1.0
-50.0
-48.0
5.0
-5.0
3.0
2.0
-3.0
1.0
-6.0
1.0
-48.0
-48.0
-48.0
-3.0
-49.0
2.0
-1.0
-49.0
-1.0
-3.0
4.0
-48.0
-6.0
-53.0
-1.0
-2.0
0.0
-1.0
2.0
-1.0
-3.0
-48.0
6.0
-3.0
3.0
-1.0
-46.0
1.0
2.0
-46.0
-49.0
4.0
4.0
-2.0
2.0
2.0
4.0
-1.0
-1.0
-49.0
-48.0
-50.0
-1.0
-1.

In [53]:
duplicate_count = item_profiles['Name'].value_counts(ascending=True)
#duplicate_count

In [54]:
Healthiness_rating_count = item_profiles['Healthiness'].value_counts()
#Healthiness_rating_count

In [ ]:
item_profiles_healthy = item_profiles2[item_profiles2.Healthiness != 0]
Healthiness_rating_count_5 = item_profiles2_healthy['Healthiness'].value_counts(normalize=False, sort=True, ascending=True)
Healthiness_rating_count_5

In [ ]:
item_profiles2_very_healthy = item_profiles2[item_profiles2.fatPoints != 0]
item_profiles2_very_healthy = item_profiles2_very_healthy[item_profiles2_very_healthy.satFatPoints != 0]
item_profiles2_very_healthy = item_profiles2_very_healthy[item_profiles2_very_healthy.sugarPoints != 0]
Healthiness_rating_count_4 = item_profiles2_very_healthy['Healthiness'].value_counts(normalize=False, sort=True, ascending=True)
Healthiness_rating_count_4

In [ ]:
#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = ratings.copy()
y = ratings['user_id']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

data = Dataset.load_from_df(ratings, reader)
cross_validate(svd, data, measures=['RMSE'])